In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import astropy.io.fits as fits
import os
from scipy.io.idl import readsav
from astropy.table import Table
from astropy.io import ascii
import astropy.units as u
import astropy.constants as const
from astropy.modeling import models, fitting
from craftroom import resample
from scipy.interpolate import interp1d
from astropy.units import cds

cds.enable()

%matplotlib inline
from matplotlib import rcParams
rcParams["figure.figsize"] = (14, 5)
rcParams["font.size"] = 20

In [2]:
muscles_path = '../../../stuff/muscles_eg/gj176/'
os.listdir(muscles_path)

['hlsp_muscles_hst_stis_gj176_g230l_v22_component-spec.fits',
 'hlsp_muscles_hst_cos_gj176_g230l_v22_component-spec.fits',
 'hlsp_muscles_model_euv-scaling_gj176_na_v22_component-spec.fits',
 'hlsp_muscles_multi_multi_gj176_broadband_v22_adapt-const-res-sed.fits',
 'hlsp_muscles_cxo_acis_gj176_na_v22_component-spec.fits',
 'hlsp_muscles_hst_cos_gj176_g130m_v22_component-spec.fits',
 'hlsp_muscles_model_apec_gj176_na_v22_component-spec.fits',
 'hlsp_muscles_multi_multi_gj176_broadband_v22_const-res-sed.fits',
 'hlsp_muscles_model_lya-reconstruction_gj176_na_v22_component-spec.fits',
 'hlsp_muscles_hst_stis_gj176_g140m_v22_component-spec.fits',
 'hlsp_muscles_hst_stis_gj176_g430l_v22_component-spec.fits',
 'hlsp_muscles_multi_multi_gj176_broadband_v22_var-res-sed.fits',
 'hlsp_muscles_multi_multi_gj176_broadband_v22_adapt-var-res-sed.fits',
 'hlsp_muscles_hst_cos_gj176_g160m_v22_component-spec.fits',
 'hlsp_muscles_model_phx_gj176_na_v22_component-spec.fits']

In [3]:
hdul = fits.open(muscles_path+'hlsp_muscles_multi_multi_gj176_broadband_v22_var-res-sed.fits')

In [4]:
hdul.info()

Filename: ../../../stuff/muscles_eg/gj176/hlsp_muscles_multi_multi_gj176_broadband_v22_var-res-sed.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      53   ()      
  1  SPECTRUM      1 BinTableHDU     59   1196260R x 13C   [D, D, D, D, D, D, I, D, D, J, D, D, D]   
  2  INSTLGND      1 BinTableHDU     32   24R x 5C   [J, 5A, 18A, 6A, 70A]   


In [5]:
hdul[2].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  103 / length of dimension 1                          
NAXIS2  =                   24 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    5 / number of table fields                         
EXTNO   =                    3                                                  
TTYPE1  = 'BITVALUE'                                                            
TFORM1  = 'J       '                                                            
TTYPE2  = 'TELESCOPE'                                                           
TFORM2  = '5A      '        

In [6]:
hdul[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
TELESCOP= 'MULTI   '                                                            
INSTRUME= 'MULTI   '                                                            
GRATING = 'MULTI   '                                                            
TELESC00= 'MODEL   '                                                            
INSTRU00= 'PHX     '                                                            
GRATIN00= 'NA      '                                                            
TELESC01= 'MODEL   '                                                            
INSTRU01= 'EUV-SCALING'                                                         
GRATIN01= 'NA      '        

In [7]:
hdul[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                   94 / length of dimension 1                          
NAXIS2  =              1196260 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   13 / number of table fields                         
EXTNAME = 'SPECTRUM'                                                            
EXTNO   =                    2                                                  
TDESC1  = 'midpoint of the wavelength bin'                                      
TDESC2  = 'left/blue edge of the wavelength bin'                                
TDESC3  = 'right/red edge of

In [8]:
hdul[2].data

FITS_rec([(      1, 'HST', 'COS', 'G130M', 'hlsp_muscles_hst_cos_gj176_g130m_v22_component-spec.fits'),
          (      2, 'HST', 'COS', 'G160M', 'hlsp_muscles_hst_cos_gj176_g160m_v22_component-spec.fits'),
          (      4, 'HST', 'COS', 'G230L', 'hlsp_muscles_hst_cos_gj176_g230l_v22_component-spec.fits'),
          (      8, 'HST', 'STIS', 'G140M', 'hlsp_muscles_hst_stis_gj176_g140m_v22_component-spec.fits'),
          (     16, 'HST', 'STIS', 'E140M', 'hlsp_muscles_hst_stis_gj176_e140m_v22_component-spec.fits'),
          (     32, 'HST', 'STIS', 'E230M', 'hlsp_muscles_hst_stis_gj176_e230m_v22_component-spec.fits'),
          (     64, 'HST', 'STIS', 'E230H', 'hlsp_muscles_hst_stis_gj176_e230h_v22_component-spec.fits'),
          (    128, 'HST', 'STIS', 'G230L', 'hlsp_muscles_hst_stis_gj176_g230l_v22_component-spec.fits'),
          (    256, 'HST', 'STIS', 'G430L', 'hlsp_muscles_hst_stis_gj176_g430l_v22_component-spec.fits'),
          (    512, 'HST', 'STIS', 'G430M', 'hlsp_mu

In [9]:
sed = Table.read('t1_table_test.ecsv')

In [10]:
instlist = np.load('ti_instlist.npy')

In [11]:
instlist

array([       1,        2,        4,     1024,    65536,        8,
            256,   131072,     2048,    32768, 16777216])

In [12]:
# instrument_list = [1, 65536, 8, 8388608, 128, 256, 131072, 8192, 32768, 16777216]

In [13]:
import instruments

In [14]:
for inst in instlist:
    print(instruments.getinststr(inst))

hst_cos_g130m
hst_cos_g160m
hst_cos_g230l
mod_gap_fill-
mod_lya_young
hst_sts_g140m
hst_sts_g430l
mod_phx_-----
xmm_epc_multi
mod_apc_-----
mod_dem_-----


In [15]:
telescopes = [instruments.getinststr(inst)[0:3] for inst in instlist]
instrus = [instruments.getinststr(inst)[4:7] for inst in instlist]
gratings = [instruments.getinststr(inst)[8:] for inst in instlist]
print(telescopes, instrus, gratings)

['hst', 'hst', 'hst', 'mod', 'mod', 'hst', 'hst', 'mod', 'xmm', 'mod', 'mod'] ['cos', 'cos', 'cos', 'gap', 'lya', 'sts', 'sts', 'phx', 'epc', 'apc', 'dem'] ['g130m', 'g160m', 'g230l', 'fill-', 'young', 'g140m', 'g430l', '-----', 'multi', '-----', '-----']


In [16]:
for inst in instlist:
    inst_str = instruments.getinststr(inst)
    a,b,c = instruments.gethlspname(inst_str)
    print(a,b,c)

HST COS G130M
HST COS G160M
HST COS G230L
MODEL POLYNOMIAL-FIT NA
MODEL LYA-RECONSTRUCTION NA
HST STIS G140M
HST STIS G430L
MODEL PHX NA
XMM EPIC MULTI
MODEL APEC NA
MODEL DEM NA


In [17]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from astropy.table import Table
from astropy.io import ascii
import astropy.units as u
from astropy.units import cds
import instruments
cds.enable()
"""
Making hslp standard fits files for Mega-Muscles
"""
def make_primary_header(hdr, sed_table, instrument_list):
    meta = sed_table.meta
    telescopes = [instruments.getinststr(inst)[0:3] for inst in instrument_list]
    instrus = [instruments.getinststr(inst)[4:7] for inst in instrument_list]
    gratings = [instruments.getinststr(inst)[8:] for inst in instrument_list]
    hdr.append(('TELESCOP', 'MULTI'))
    hdr.append(('INSTRUME', 'MULTI'))
    hdr.append(('GRATING', 'MULTI'))
    for i in range(len(telescopes)):
        hdr.append(('TELESC{:02.0f}'.format(i), telescopes[i].upper()))
        hdr.append(('INSTRU{:02.0f}'.format(i), instrus[i].upper()))
        hdr.append(('GRATIN{:02.0f}'.format(i), gratings[i].upper()))
    extra_keys =  ['TARGNAME','RA_TARG','DEC_TARG','PROPOSID','HLSPNAME','HLSPACRN','HLSPLEAD','PR_INV_L',
                   'PR_INV_F','WAVEMIN','WAVEMAX','WAVEUNIT','AIRORVAC','FLUXMIN',
                  'FLUXMAX','FLUXUNIT', 'BOLOFLUX', 'LNZ_NORM','LNZ_GAM']
    for key in extra_keys[:-5]: #temporary while I don't have the last ones in there.
        hdr.append((key,meta[key]))
    return hdr



def data_header(hdr):
    """
    Makes the header for the table extension. 
    
    Keywords to add from MUSCLES:

    TDESC1  = 'midpoint of the wavelength bin'                                      
    TDESC2  = 'left/blue edge of the wavelength bin'                                
    TDESC3  = 'right/red edge of the wavelength bin'                                
    TDESC4  = 'average flux over the bin'                                           
    TDESC5  = 'error on the flux'                                                   
    TDESC6  = 'cumulative exposure time for the bin'                                
    TDESC7  = 'data quality flags (HST data only)'                                  
    TDESC8  = 'modified julian date of start of first exposure'                     
    TDESC9  = 'modified julian date of end of last exposure'                        
    TDESC10 = 'bitmask identifying the source instrument(s). See "instlgnd" & extension for a legend.'                                             
    TDESC11 = 'normalization factor applied to the source spectrum'                 
    TDESC12 = 'flux density normalized by the bolometric flux'                      
    TDESC13 = 'error on bolometrically-normalized flux density'

    """
    new_keywords = ('TDESC1','TDESC2','TDESC3','TDESC4', 'TDESC5',
               'TDESC6', 'TDESC7', 'TDESC8', 'TDESC9', 'TDESC10',
               'TDESC11','TDESC12','TDESC13') 
    new_values =  ('midpoint of the wavelength bin',
                   'left/blue edge of the wavelength bin',
                   'right/red edge of the wavelength bin',
                   'average flux over the bin',
                   'error on the flux',
                   'cumulative exposure time for the bin',
                   'data quality flags (HST data only)',
                   'modified julian date of start of first exposure',
                   'modified julian date of end of last exposure',
                   'bitmask identifying the source instrument(s). See "instlgnd" & extension for a legend.',
                   'normalization factor applied to the source spectrum',
                   'flux density normalized by the bolometric flux',
                   'error on bolometrically-normalized flux density')
    for i, n, v in zip(range(len(new_keywords)), new_keywords, new_values):
        hdr.insert(i+8, (new_keywords[i], new_values[i]))
    
    return hdr


def make_data_ext(sed_table):
    """
    The table extension, takes an astropy table 
    """
    hdu = fits.table_to_hdu(Table(dict(sed_table)))
    hdu.header = data_header(hdu.header)
    return hdu


def make_primary_ext(sed_table, instrument_list):
    """
    Make the primary header
    """
    primary_hdu = fits.PrimaryHDU()
    primary_hdu.header = make_primary_header(primary_hdu.header, sed_table, instrument_list)
    return primary_hdu

# def make_mm_fits(savepath, sed_table, instrument_list, version,sed_type='var'):
#     """
#     Saves an SED as a Mega-MUSCLES fits file
#     """
#     primary_hdu = make_primary_ext(sed_table, instrument_list)
#     data_ext = make_data_ext(sed_table)
#     #add instruments here
#     star = sed_table.meta['TARGNAME'].lower()
#     if star == '2mass-j23062928-0502285':
#         star = 'trappist-1'
#     file_name = 'hlsp_muscles_multi_multi_{}_broadband_v{}_{}-res-sed'.format(star, version, sed_type)
#     hdul = fits.HDUList([primary_hdu, data_ext])
#     hdul.writeto('{}{}.fits'.format(savepath,file_name), overwrite=True)
#     print('sed saved as {}'.format(file_name))

In [18]:
# import make_fits

In [19]:
testdata = make_data_ext(Table(dict(sed)))
testdata.header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  104 / length of dimension 1                          
NAXIS2  =              1178063 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   13 / number of table fields                         
TDESC1  = 'midpoint of the wavelength bin'                                      
TDESC2  = 'left/blue edge of the wavelength bin'                                
TDESC3  = 'right/red edge of the wavelength bin'                                
TDESC4  = 'average flux over the bin'                                           
TDESC5  = 'error on the flux

In [20]:
hdul[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
TELESCOP= 'MULTI   '                                                            
INSTRUME= 'MULTI   '                                                            
GRATING = 'MULTI   '                                                            
TELESC00= 'MODEL   '                                                            
INSTRU00= 'PHX     '                                                            
GRATIN00= 'NA      '                                                            
TELESC01= 'MODEL   '                                                            
INSTRU01= 'EUV-SCALING'                                                         
GRATIN01= 'NA      '        

In [21]:
# plt.plot(testdata.data['WAVELENGTH'], testdata.data['FLUX'])

In [22]:
# def make_primary_header(hdr, sed_table, instrument_list):
#     meta = sed_table.meta
#     telescopes = [instruments.getinststr(inst)[0:3] for inst in instrument_list]
#     instrus = [instruments.getinststr(inst)[4:7] for inst in instrument_list]
#     gratings = [instruments.getinststr(inst)[8:] for inst in instrument_list]
#     hdr.append(('TELESCOP', 'MULTI'))
#     hdr.append(('INSTRUME', 'MULTI'))
#     hdr.append(('GRATING', 'MULTI'))
#     for i in range(len(telescopes)):
#         hdr.append(('TELESC{:02.0f}'.format(i), telescopes[i].upper()))
#         hdr.append(('INSTRU{:02.0f}'.format(i), instrus[i].upper()))
#         hdr.append(('GRATIN{:02.0f}'.format(i), gratings[i].upper()))
#     extra_keys =  ['TARGNAME','RA_TARG','DEC_TARG','PROPOSID','HLSPNAME','HLSPACRN','HLSPLEAD','PR_INV_L',
#                    'PR_INV_F','WAVEMIN','WAVEMAX','WAVEUNIT','AIRORVAC','FLUXMIN',
#                   'FLUXMAX','FLUXUNIT', 'BOLOFLUX', 'LNZ_NORM','LNZ_GAM']
#     for key in extra_keys[:-2]: #temporary while I don't have the last ones in there.
#         hdr.append((key,meta[key]))
#     return hdr
        
# testpri = make_primary_ext()
# testprihdr = make_primary_header(testpri.header, sed, instlist)
# testprihdr


TypeError: make_primary_ext() missing 2 required positional arguments: 'sed_table' and 'instrument_list'

In [ ]:
# sed.meta

In [ ]:
# 'help{:02.0f}'.format(10)

In [ ]:
# sed['BOLOFLUX']

In [23]:
hdul[2].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  103 / length of dimension 1                          
NAXIS2  =                   24 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    5 / number of table fields                         
EXTNO   =                    3                                                  
TTYPE1  = 'BITVALUE'                                                            
TFORM1  = 'J       '                                                            
TTYPE2  = 'TELESCOPE'                                                           
TFORM2  = '5A      '        

So we can make the instrument file out of one of the MUSCLES ones. Oh no they are differnt.

In [26]:
inst_tab = Table(hdul[2].data, meta=hdul[2].header)

BITVALUE,TELESCOPE,INSTRUMENT,GRATING,HLSP_FILE
int32,str5,str18,str6,str70
1,HST,COS,G130M,hlsp_muscles_hst_cos_gj176_g130m_v22_component-spec.fits
2,HST,COS,G160M,hlsp_muscles_hst_cos_gj176_g160m_v22_component-spec.fits
4,HST,COS,G230L,hlsp_muscles_hst_cos_gj176_g230l_v22_component-spec.fits
8,HST,STIS,G140M,hlsp_muscles_hst_stis_gj176_g140m_v22_component-spec.fits
16,HST,STIS,E140M,hlsp_muscles_hst_stis_gj176_e140m_v22_component-spec.fits
32,HST,STIS,E230M,hlsp_muscles_hst_stis_gj176_e230m_v22_component-spec.fits
64,HST,STIS,E230H,hlsp_muscles_hst_stis_gj176_e230h_v22_component-spec.fits
128,HST,STIS,G230L,hlsp_muscles_hst_stis_gj176_g230l_v22_component-spec.fits
256,HST,STIS,G430L,hlsp_muscles_hst_stis_gj176_g430l_v22_component-spec.fits


In [29]:
# adding new MM rows
inst_tab.add

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  103 / length of dimension 1                          
NAXIS2  =                   24 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    5 / number of table fields                         
EXTNO   =                    3                                                  
TTYPE1  = 'BITVALUE'                                                            
TFORM1  = 'J       '                                                            
TTYPE2  = 'TELESCOPE'                                                           
TFORM2  = '5A      '        